In [65]:
# -*- coding: utf-8 -*- 
from __future__ import unicode_literals

import numpy as np
import pandas as pd
import json
import itertools as itt
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns



import geopandas as gpd
import shapely
import fiona
import CoordinatesConverter as convert_xy  #the .py file for bd09 conversioin

%matplotlib inline
#plt.style.use('ggplot')
plt.rcParams['figure.figsize']=[18,8]
plt.rcParams['axes.titlesize'] = 'medium'
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['legend.frameon'] = False


WGS84 = 'EPSG:4326'
#UTM_bj = ccrs.UTM(50)
UTM_bj = 'EPSG:32650'
UTM_nj = 'EPSG:32650'


## set the projection (planar) crs of this area
CRS_planar = UTM_nj

# loading data

## overall boundary
the maximum extent for  blocks generation

In [2]:
boundary = gpd.read_file(r'D:\Research\Some Basic Data\Nanjing\cityborder_2035').to_crs(CRS_planar)
boundary = boundary.geometry.explode(index_parts=True).reset_index(drop=True).reset_index()
boundary['geometry'] = [shapely.geometry.Polygon(i) for i in boundary['geometry']]
boundary['geometry'] = boundary.buffer(0)

## landuse

In [ ]:
landuse = gpd.read_file(r'D:\Research\Some Basic Data\Nanjing\landuse 2015\new landuse_nj.shp').to_crs(CRS_planar)

#landuse = landuse[~landuse['类型'].isin(['水域'])]
s = gpd.sjoin(landuse, boundary)
landuse = landuse[landuse.index.isin(s.index)].copy()

# generate blocks

## merge to get blocks

In [10]:
# merge to get blocks
s = landuse.unary_union
blocks = gpd.GeoSeries(list(s.geoms)).reset_index()
blocks.columns = ['blockID', 'geometry']
blocks = gpd.GeoDataFrame(blocks,crs = landuse.crs)

In [12]:
blocks.to_file('../data/blocks_from_new_landuse')

## purging
### delete some blocks outside boundary manually in QGIS 

In [101]:
## 1. in QGIS delete some blocks outside boundary manually
blocks = gpd.read_file(r'D:\Research\Some Basic Data\Nanjing\block_from_landuse')

In [13]:
blocks2 = blocks.to_crs(CRS_planar)

### remove blocks that are too small

In [ ]:
## 2. remove blocks that are too small
min_width = 50
blocks2 = blocks2[[not i.buffer(-min_width/2).is_empty for i in blocks2['geometry']]]

### fix small cracks

In [14]:
## 3. fix small cracks
blocks2['geometry'] = blocks2.buffer(1).buffer(-1)

### merge blocks that are supposed to be merged but are not due to small cracks now

In [15]:
## 4. merge blocks that are supposed to be merged but are not due to small cracks now
blocks3 = pd.Series(list(blocks2.buffer(1).unary_union)).reset_index()
blocks3.columns = ['blockID', 'geometry']
blocks3 = gpd.GeoDataFrame(blocks3, crs=blocks2.crs)
blocks3['geometry'] = blocks3.buffer(-1)

#blocks3 = blocks3.to_crs(WGS84)

In [16]:
blocks3.to_file(r'D:\Research\Some Basic Data\Nanjing\block_from_new_landuse_purged')